In [1]:
import numpy 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
#loads ascii text and convert to lowercase
filename = "shakespeare.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [7]:
# create mapping of unique chars to integer
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

In [10]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ",n_chars)
print ("Total Vocab: ",n_vocab)

Total Characters:  5668982
Total Vocab:  67


In [12]:
# prepare thre dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range (0 , n_chars - seq_length, 1):
    seq_in = raw_text[i: i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patters: ", n_patterns)

Total Patters:  5668882


In [13]:
# reshape X to be [sample, time steps, features]
X = numpy.reshape(dataX, (n_patters, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot coding the output variable
y = np_utils.to_categorical(dataY)

In [17]:
# define LSTM model
# from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()
# plot_model(model, to_file='LSTM_text_generation_model_plot.png', show_shapes=True, show_layer_names=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 67)                17219     
Total params: 281,411
Trainable params: 281,411
Non-trainable params: 0
_________________________________________________________________


In [18]:
filepath="weighted-improvement-{epoch:02d}-{loss:4f}.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='loss',
                           verbose=1,
                           save_best_only=True,
                           mode='min')
callbacks_list=[checkpoint]

In [19]:
epochs = 10
model.fit(X, y, epochs=epochs, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
 544384/5668882 [=>............................] - ETA: 46586s - loss: 2.6957

KeyboardInterrupt: 

In [ ]:
# load the network weights
filename = ""
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
int_to_char=dict((i,c)for i,c in enumerate(chars))

In [ ]:
# pick random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed: ")
print("\", ''.join([int_to_char[value] for value in pattern]), "\)
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\n DONE PREDICTION")